In [1]:
import os
from osgeo import gdal, ogr
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
from rasterio.mask import mask

In [2]:
in_tif=r'P:\Temp\sputnam\data\rock_creek_clip.tif'

In [4]:
stream_network = rasterio.open(in_tif)

In [6]:
class StreamGrid(object):
    '''FlowAccGrid class designed to read in flow accumulation grids from the NY USGS
       Streamstats geodatabase.
       Documentation & additianl info for gdal found here
       https://pypi.python.org/pypi/GDAL/
       https://pcjericks.github.io/py-gdalogr-cookbook/'''

    def __init__(self,path):
        self.path = path
        self.data = gdal.Open(path)
        self.band = self.data.GetRasterBand(1)
        self.array= self.band.ReadAsArray()

    def dataframe(self):
        '''Returns the flow accumulation grid as a pandas dataframe object'''
        df = pd.DataFrame(self.array)
        return df

    def transform(self,x,y):
        '''Method takes indices from dataframe or array and returns projected coordinates'''
        ul_x, x_dim, x_rotation, ul_y, y_rotation, y_dim = self.data.GetGeoTransform()
        x_coord = x * x_dim + ul_x + (x_dim / 2.)
        y_coord = y * y_dim + ul_y + (y_dim / 2.)
        coords = x_coord, y_coord
        return coords

In [ ]:
def MoveUpstream(df, row, col, nogo_cells):
    '''Function searches surrounding cells in stream network grid to identify the
        grid cell immediately upstream. Output is index pair of upstream cell '''
    curr_cell_idxs = (row, col)
    
    current_cell= df[row][col]
    assert current_cell.values == 1
    surr_cells = []

    for i in range(-1,2):
        
        for j in range(-1,2):

            value = df[row + i][col+j] #Read Value of raster cell

            if value == 0:
                # if value is zero, no stream in this cell
                continue
                
            elif value == 1:
                # if value is 1, add to list
                surr_cells.append([row + i],[col+j])
                
            elif (row, col) in nogo_cells:
                continue
                
            else:
                print("Error?")
                
    return curr_cell_idxs, surr_cells

In [ ]:
def UpstreamIterator(flowgrid, idx_n, idx_list,str_len):
    '''Algorithm created to iterate over steps to calculate streamline distance and find the
       uus point of interest (stopping point, defined by distance given in while expression in __main__) '''
    r,c = int(idx_n[0][0]),int(idx_n[1][0])
    idx_n = MoveUpstream(flowgrid.dataframe(),r,c)
    xypair = idx_n[0][0],idx_n[1][0]
    str_len += GetDistance(r,c, xypair[0], xypair[1], cellsize)
    usxy = flowgrid.transform(xypair[0],xypair[1])
    idx_list.append(usxy)
    return usxy, idx_n, str_len

In [7]:
sg = StreamGrid(in_tif)

In [13]:
df = sg.dataframe()
df.replace(255,0, inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1205,1206,1207,1208,1209,1210,1211,1212,1213,1214
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
